# Latihan-12 ThinChannel

Latihan ini untuk pemodelan 2D model untuk visualisasi perubahan ketebalan geometri seperti channel-like feature
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-10 2D Kue Lapis, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from scipy.signal import hilbert
import scipy as sp
import scipy.ndimage as snd
import copy
from matplotlib import cm
from f10kuelapis import *
mpl.style.use('seaborn') # default, seaborn

In [ ]:
#---------------- tweaks dari wedge model menjadi simple channel-like feature-------------------
n_samples, n_traces = [500, 1000]
rock_names = ['shale1', 'sand', 'shale2']
rock_gridl = np.zeros((n_samples, n_traces))
rock_gridr = np.zeros((n_samples, n_traces))
def make_wedge(n_samples, n_traces, h1, h2, start_wedge, cut_half):
    for j in np.arange(n_traces): 
        for i in np.arange(n_samples):      
            if i <= h1:      
                rock_gridl[i][j] = 1    
            if i > h1:         
                rock_gridl[i][j] = 3    
                if j > start_wedge and j <=cut_half : 
                    if i - h1 < j-start_wedge and i - h1 > j - cut_half :
                        rock_gridl[i][j] = 2
                        if j >= h2:
                            if i > h1+(h2-start_wedge) :
                                rock_gridl[i][j] = 3
    rock_gridr=np.fliplr(rock_gridl)
    rock_grid=rock_gridl+rock_gridr
    return rock_grid
h1 = 200
h2 = 100 # paling tipis 30unit utk bisa terlihat
start_wedge = 20
cut_half = 499
rock_grid = make_wedge(n_samples, n_traces, h1, h2, start_wedge, cut_half)
plt.figure(figsize=(10,8))
im=plt.imshow(rock_grid, cmap='Spectral', aspect=1)
plt.colorbar(im,fraction=0.02, pad=0.02)
plt.title('Empty Vessel', fontsize=16)
plt.show()

In [ ]:
print(rock_grid)

In [ ]:
#------model acoustic impedance-------------
ip = np.array([24000, 8000, 20000])
model = np.copy(rock_grid)
model[rock_grid == 2] = ip[0]
model[rock_grid == 4] = ip[1]
model[rock_grid == 6] = ip[2]
plt.figure(figsize=(10,8))
im=plt.imshow(model, cmap='rainbow', aspect=1)
plt.colorbar(im,fraction=0.02, pad=0.02)
plt.title('Acoustic Impedance', fontsize=16)
plt.show()

In [ ]:
upper = model[:-1][:]
lower = model[1:][:]
rc = (lower - upper) / (lower + upper)
maxrc = abs(np.amax(rc))
plt.figure(figsize=(10,8))
im=plt.imshow(rc, cmap='bwr', aspect=1)
plt.colorbar(im,fraction=0.02, pad=0.01)
plt.title('Reflectivity (Zero Incident)', fontsize=16)
plt.grid(False)
plt.show()

In [ ]:
def ricker(f, l, dt, wphase):
    t = np.linspace(-l/2, (l-dt)/2, int(l/dt))/1000
    zeroph = (1.0 - 2.0*(np.pi**2)*(f**2)*(t**2)) * np.exp(-(np.pi**2)*(f**2)*(t**2))
    h = hilbert(zeroph)
    rotdeg = wphase
    theta = rotdeg*np.pi/180
    y = np.cos(theta)*h.real-np.sin(theta)*h.imag
    return t, y
t,wavelet=ricker(20,120,0.4,0)
plt.figure(figsize=(4,2))
plt.plot(t,wavelet, 'k', lw=1)
plt.fill_between(t,wavelet,where=(wavelet>0),facecolor='b', alpha=0.3)
plt.fill_between(t,wavelet,where=(wavelet<=0),facecolor='r', alpha=0.3)
plt.grid(color = 'gray', linestyle = ':', linewidth = 0.6)
plt.xlim([-0.1, 0.1]), plt.ylim([-0.5, 1]); plt.axis('off')
plt.show()

In [ ]:
synth = np.apply_along_axis(lambda t: np.convolve(t, wavelet, mode='full'), axis=0, arr=rc)
plt.figure(figsize=(20,8))
im=plt.imshow(synth, cmap="bwr", aspect=1)
plt.colorbar(im,fraction=0.02, pad=0.05)
plt.xlim([0, 1000])
plt.grid(False)
plt.show()

#  <h1><center>-Akhir Program Latihan-12-<center></h1>